# 데이터 전처리(분류, 회귀, 군집화 모두 똑같음)

## 데이터 불러오기 

In [ ]:
import pandas as pd

# Load the dataset
file_path = '파일 경로로'
data = pd.read_csv(file_path)

# Display the first few rows of the dataset to understand its structure
data.head()


### 1. 결측값 처리

In [ ]:
# 결측값 확인
print(data.isnull().sum())

# 수치형 열의 결측값을 해당 열의 평균으로 대체
data['horsepower'].fillna(data['horsepower'].mean(), inplace=True)
data['mpg'].fillna(data['mpg'].mean(), inplace=True)

# 텍스트형 열의 결측값을 "Unknown"으로 대체
data['origin'].fillna('Unknown', inplace=True)


# 결측값 50%이상인 행 삭제
# 각 행의 결측값 비율 계산
row_missing_percentage = data.isnull().mean(axis=1) * 100

# 결측값 비율이 50% 이상인 행 삭제
rows_to_drop = row_missing_percentage[row_missing_percentage >= 50].index
data.drop(index=rows_to_drop, inplace=True)

# 변경된 데이터 확인
print(data.isnull().sum())



### 2. 범주형 변수 처리

In [ ]:
# 'origin' 열을 원-핫 인코딩
data = pd.get_dummies(data, columns=['origin'], drop_first=True)




from sklearn.preprocessing import LabelEncoder

# 'origin' 열을 라벨 인코딩 (1, 2, 3으로 변환)
label_encoder = LabelEncoder()
data['origin'] = label_encoder.fit_transform(data['origin'])


### 3. 스케일링

In [ ]:
#표준화
from sklearn.preprocessing import StandardScaler

# 수치형 데이터 열만 적기
numeric_cols = ['mpg', 'horsepower', 'weight', 'acceleration', 'model year', 'cylinders']

# 표준화
scaler = StandardScaler()
data[numeric_cols] = scaler.fit_transform(data[numeric_cols])

print(data.head())

In [ ]:
#정규화
from sklearn.preprocessing import MinMaxScaler

# 정규화
scaler = MinMaxScaler()
data[numeric_cols] = scaler.fit_transform(data[numeric_cols])

print(data.head())


### 5. 이상치 처리

In [ ]:
# IQR을 이용해 이상치 탐지
Q1 = data['horsepower'].quantile(0.25)
Q3 = data['horsepower'].quantile(0.75)
IQR = Q3 - Q1

# 이상치 경계
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# 이상치가 아닌 값만 필터링
data_no_outliers = data[(data['horsepower'] >= lower_bound) & (data['horsepower'] <= upper_bound)]

print(data_no_outliers.head())


### 6. 데이터 분할

In [ ]:
from sklearn.model_selection import train_test_split

# 특성과 타겟 변수 분리
# 타겟변수는 모델이 예측해야 하는 결과값
X = data.drop(columns=['mpg'])  # 타겟 변수 'mpg' 제외
y = data['mpg']

# 데이터 분할 (훈련용 80%, 테스트용 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train.shape, X_test.shape)


### 7. 텍스트형 데이터 전처리
 

In [ ]:
import re

# 예시 텍스트 열 처리 (가정: 'car_name' 열이 텍스트형 데이터)
def preprocess_text(text):
    text = text.lower()  # 소문자 변환
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # 특수 문자 제거
    return text

data['car_name'] = data['car_name'].apply(preprocess_text)

print(data['car_name'].head())

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 텍스트 데이터를 TF-IDF 벡터로 변환
tfidf = TfidfVectorizer()
X_text = tfidf.fit_transform(data['car_name'])

print(X_text.shape)  # 벡터화된 데이터의 크기

### 8. 데이터 시각화

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 'mpg'와 'horsepower'의 관계 시각화
sns.scatterplot(x=data['horsepower'], y=data['mpg'])
plt.title('Horsepower vs MPG')
plt.xlabel('Horsepower')
plt.ylabel('MPG')
plt.show()


In [ ]:
## 회귀 데이터 선형성 확인을 위한 산점도 그리기
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 데이터 불러오기
file_path = 'auto-mpg.csv'
data = pd.read_csv(file_path)

# 특성과 타겟 변수 선택
X = data['horsepower'] #특성
y = data['mpg'] # 타겟변수

# 선형 관계 산점도
plt.figure(figsize=(8, 6))
plt.scatter(X, y, color='blue', alpha=0.7)
plt.title("Scatter Plot: Horsepower vs MPG (Linear)")
plt.xlabel("Horsepower")
plt.ylabel("MPG")
plt.show()

# 비선형 관계 산점도 예시 (weight vs mpg)
X_nonlinear = data['weight']
y_nonlinear = data['mpg']

plt.figure(figsize=(8, 6))
plt.scatter(X_nonlinear, y_nonlinear, color='red', alpha=0.7)
plt.title("Scatter Plot: Weight vs MPG (Nonlinear)")
plt.xlabel("Weight")
plt.ylabel("MPG")
plt.show()


### 차원 축소

In [ ]:
from sklearn.decomposition import PCA

# PCA를 사용한 차원 축소
pca = PCA(n_components=2)
data_reduced = pca.fit_transform(data)
